# Change Rankings

In [3]:
from pref_voting.profiles import Profile
from pref_voting.profiles_with_ties import ProfileWithTies
from pref_voting.generate_profiles import generate_profile, generate_truncated_profile
from pref_voting.rankings import Ranking

In [16]:
prof = generate_profile(4, 10)

print(prof)

prof = prof.to_profile_with_ties()
prof = ProfileWithTies([
    {0:1},
    {0:1, 1:2},
    {0:1, 1:2, 2:3},
    {1:1, 2:2, 0:3},
    {2:1, 1:1, 0:3}
],
rcounts=[3, 5, 4, 1, 2], candidates = [0, 1, 2])
prof.display()
print(prof.num_bullet_votes())
print(prof.num_linear_orders())
print(prof.num_truncated_linear_orders())
print(prof.num_rankings_with_ties())
print(prof.num_ranked_all_candidates())
print(prof.num_ranking_each_candidate())

+---+---+---+---+---+---+---+---+---+---+
| 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+---+---+---+---+
| 2 | 0 | 2 | 2 | 0 | 3 | 1 | 2 | 2 | 2 |
| 3 | 2 | 3 | 3 | 1 | 2 | 3 | 1 | 1 | 0 |
| 0 | 1 | 0 | 1 | 3 | 0 | 0 | 3 | 3 | 1 |
| 1 | 3 | 1 | 0 | 2 | 1 | 2 | 0 | 0 | 3 |
+---+---+---+---+---+---+---+---+---+---+
+---+---+---+---+-----+
| 3 | 5 | 4 | 1 |  2  |
+---+---+---+---+-----+
| 0 | 0 | 0 | 1 | 2 1 |
|   | 1 | 1 | 2 |  0  |
|   |   | 2 | 0 |     |
+---+---+---+---+-----+
3
5
8
2
7
{0: 15, 1: 12, 2: 7}


In [ ]:
def same_ranking_extended_strict_pref(ranking1, ranking2, candidates): 
    # check if ranking1 and ranking2 have the same ranking of candidates
    for c1 in candidates:
        for c2 in candidates:
            if (not ranking1.extended_strict_pref(c1, c2) and ranking2.extended_strict_pref(c1, c2)) or (not ranking2.extended_strict_pref(c1, c2) and ranking1.extended_strict_pref(c1, c2)):
                return False
    return True

def replace_rankings(
        profile, 
        old_ranking, 
        new_ranking, 
        num, 
        use_extended_strict_preference_for_comparison = False): 
    """

    Create a new profile by replacing num ballots matching ranking1 with ranking2.

    If num is greater than the number of ballots matching ranking1, then all ballots matching ranking1 are replaced with ranking2.
    
    """
    is_profile = isinstance(profile, Profile)
    is_profile_with_ties = isinstance(profile, ProfileWithTies)

    if is_profile_with_ties:
        using_extended_strict_pref = profile.using_extended_strict_preference
    
    ranking_types, ranking_counts = profile.rankings_counts

    if is_profile_with_ties: 
        if not isinstance(old_ranking, Ranking) or not isinstance(new_ranking, Ranking):
            raise ValueError("rankings must be of type Ranking")
        
    if is_profile: 
        same_ranking = lambda r1, r2: list(r1) == list(r2)
    elif is_profile_with_ties:
        if use_extended_strict_preference_for_comparison:
            same_ranking = lambda r1, r2: same_ranking_extended_strict_pref(r1, r2, profile.candidates)
        else:
            same_ranking = lambda r1, r2: r1 == r2
        
    new_ranking_types = []
    new_ranking_counts = []

    current_num = 0
    for r, c in zip(ranking_types, ranking_counts):
        
        if current_num < num and same_ranking(r, old_ranking):
            if c > num - current_num:
                new_ranking_types.append(new_ranking)
                new_ranking_counts.append(num - current_num)
                new_ranking_types.append(old_ranking)
                new_ranking_counts.append(c - (num - current_num))
                current_num = num
            elif c == num - current_num and same_ranking(r, old_ranking):
                new_ranking_types.append(new_ranking)
                new_ranking_counts.append(num - current_num)
                current_num = num
            elif c < num - current_num:
                new_ranking_types.append(new_ranking)
                new_ranking_counts.append(c)
                current_num += c
        else:
            new_ranking_types.append(r)
            new_ranking_counts.append(c)

    if is_profile:
        new_prof = Profile(new_ranking_types, new_ranking_counts, cmap=profile.cmap)
    elif is_profile_with_ties:
        new_prof = ProfileWithTies(new_ranking_types, new_ranking_counts, profile.candidates, cmap=profile.cmap)

        if using_extended_strict_pref:
            new_prof.use_extended_strict_preference()
    
    assert profile.num_voters == new_prof.num_voters, "Problem: the number of voters is not the same in the new profile!"
    
    return new_prof

In [97]:
prof = ProfileWithTies([
    Ranking({0:1, 1:2}),
    Ranking({0:2}),
    Ranking({0:1, 1:1}),
    Ranking({0:1, 1:2}),
    Ranking({0:1, 1:2, 2:3}),
],
rcounts=[2, 1, 1, 2, 1])
prof.display()  
r1 = Ranking.from_linear_order((0, 1))
print(r1)
new_prof = replace_rankings(prof, r1, Ranking.from_linear_order((2, 1)), 10, use_extended_strict_preference_for_comparison=False)

new_prof.display()

+---+---+-----+---+---+
| 2 | 1 |  1  | 2 | 1 |
+---+---+-----+---+---+
| 0 | 0 | 0 1 | 0 | 0 |
| 1 |   |     | 1 | 1 |
|   |   |     |   | 2 |
+---+---+-----+---+---+
0 1 
+---+---+-----+---+---+
| 2 | 1 |  1  | 2 | 1 |
+---+---+-----+---+---+
| 2 | 0 | 0 1 | 2 | 0 |
| 1 |   |     | 1 | 1 |
|   |   |     |   | 2 |
+---+---+-----+---+---+


In [101]:
#prof = generate_truncated_profile(3, 10).anonymize()

prof.display()  
r1 = Ranking.from_linear_order((0, 1))
print(r1)
new_prof = replace_rankings(prof, r1, Ranking.from_linear_order((2, 1)), 1, use_extended_strict_preference_for_comparison=True)

new_prof.display()

+---+---+---+---+---+---+---+---+
| 2 | 1 | 1 | 2 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+---+---+
| 0 | 2 | 1 | 0 | 1 | 1 | 1 | 2 |
|   | 0 | 2 | 1 | 0 |   | 0 | 1 |
|   | 1 | 0 |   |   |   | 2 |   |
+---+---+---+---+---+---+---+---+
0 1 
+---+---+---+---+---+---+---+---+---+
| 2 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |
+---+---+---+---+---+---+---+---+---+
| 0 | 2 | 1 | 2 | 0 | 1 | 1 | 1 | 2 |
|   | 0 | 2 | 1 | 1 | 0 |   | 0 | 1 |
|   | 1 | 0 |   |   |   |   | 2 |   |
+---+---+---+---+---+---+---+---+---+


In [106]:
prof = generate_profile(3, 10).anonymize()


prof.display()  

new_prof = replace_rankings(prof, prof.rankings[0], (2, 1, 0), 1)

new_prof.display()

+---+---+---+---+---+
| 1 | 2 | 2 | 2 | 3 |
+---+---+---+---+---+
| 1 | 1 | 0 | 2 | 2 |
| 0 | 2 | 1 | 0 | 1 |
| 2 | 0 | 2 | 1 | 0 |
+---+---+---+---+---+
+---+---+---+---+---+
| 1 | 2 | 2 | 2 | 3 |
+---+---+---+---+---+
| 2 | 1 | 0 | 2 | 2 |
| 1 | 2 | 1 | 0 | 1 |
| 0 | 0 | 2 | 1 | 0 |
+---+---+---+---+---+
